In [5]:
import numpy as np
import pandas as pd
import json
from math import radians, cos, sin, asin, sqrt

In [6]:
def haversine(lon1, lat1, lon2, lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # 将十进制度数转化为弧度
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
 
    # haversine公式
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # 地球平均半径，单位为公里
    return c * r * 1000

# train

In [7]:
df = pd.read_csv('train.csv')

In [4]:
df.shape

(1710670, 9)

In [10]:
df[df['POLYLINE'] == '[]'].shape

(5901, 9)

In [3]:
df = df[df['MISSING_DATA'] == False]
df[df['POLYLINE'] != '[]']

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1704759 entries, 0 to 1710669
Data columns (total 9 columns):
TRIP_ID         int64
CALL_TYPE       object
ORIGIN_CALL     float64
ORIGIN_STAND    float64
TAXI_ID         int64
TIMESTAMP       int64
DAY_TYPE        object
MISSING_DATA    bool
POLYLINE        object
dtypes: bool(1), float64(2), int64(3), object(3)
memory usage: 118.7+ MB


In [5]:
df.head(3)

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."


In [6]:
df['POLYLINE'] = df['POLYLINE'].apply(json.loads)
df['duration'] = ((df['POLYLINE'].apply(len))-1)*15

In [13]:
df = df.reset_index()

In [14]:
df.shape[0]

1704759

In [15]:
start = []
destination = []
for i in range(df.shape[0]):
    start.append(df['POLYLINE'][i][0])
    destination.append(df['POLYLINE'][i][-1])
df['start'] = start    
df['destination'] = destination

In [16]:
df.head()

,index,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,duration,start,destination
0,0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643, 41.141412], [-8.618499, 41.141376...",330,"[-8.618643, 41.141412]","[-8.630838, 41.154489]"
1,1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847, 41.159826], [-8.640351, 41.159871...",270,"[-8.639847, 41.159826]","[-8.66574, 41.170671]"
2,2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964, 41.140359], [-8.613378, 41.14035]...",960,"[-8.612964, 41.140359]","[-8.61597, 41.14053]"
3,3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678, 41.151951], [-8.574705, 41.151942...",630,"[-8.574678, 41.151951]","[-8.607996, 41.142915]"
4,4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994, 41.18049], [-8.645949, 41.180517]...",420,"[-8.645994, 41.18049]","[-8.687268, 41.178087]"


In [24]:
df.shape

(1704759, 13)

In [22]:
havers = []
for i in range(df.shape[0]):
    havers.append(haversine(df['start'][i][0], df['start'][i][1], df['destination'][i][0], df['destination'][i][1])) 
df['haversine'] = havers    

# 整理好的檔案

In [26]:
df.head()

,index,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,duration,start,destination,haversine
0,0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643, 41.141412], [-8.618499, 41.141376...",330,"[-8.618643, 41.141412]","[-8.630838, 41.154489]",1776.807571
1,1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847, 41.159826], [-8.640351, 41.159871...",270,"[-8.639847, 41.159826]","[-8.66574, 41.170671]",2480.360461
2,2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964, 41.140359], [-8.613378, 41.14035]...",960,"[-8.612964, 41.140359]","[-8.61597, 41.14053]",252.441979
3,3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678, 41.151951], [-8.574705, 41.151942...",630,"[-8.574678, 41.151951]","[-8.607996, 41.142915]",2965.198721
4,4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994, 41.18049], [-8.645949, 41.180517]...",420,"[-8.645994, 41.18049]","[-8.687268, 41.178087]",3464.589323


# save txt

In [32]:
df.to_csv(r'train.txt')

In [33]:
textdf = pd.read_csv('train.txt')

In [34]:
textdf.head()

,Unnamed: 0,index,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,duration,start,destination,haversine
0,0,0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643, 41.141412], [-8.618499, 41.141376...",330,"[-8.618643, 41.141412]","[-8.630838, 41.154489]",1776.807571
1,1,1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847, 41.159826], [-8.640351, 41.159871...",270,"[-8.639847, 41.159826]","[-8.66574, 41.170671]",2480.360461
2,2,2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964, 41.140359], [-8.613378, 41.14035]...",960,"[-8.612964, 41.140359]","[-8.61597, 41.14053]",252.441979
3,3,3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678, 41.151951], [-8.574705, 41.151942...",630,"[-8.574678, 41.151951]","[-8.607996, 41.142915]",2965.198721
4,4,4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994, 41.18049], [-8.645949, 41.180517]...",420,"[-8.645994, 41.18049]","[-8.687268, 41.178087]",3464.589323


# test

In [20]:
test = pd.read_csv('test.csv')
op1 = pd.read_csv('output1.csv')

In [21]:
type(test.POLYLINE[0])

str

In [22]:
test['POLYLINE'] = test['POLYLINE'].apply(json.loads)
test['duration'] = ((test['POLYLINE'].apply(len))-1)*15

In [28]:
test.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,duration
0,T1,B,NaN,15.0,20000542,1408039037,A,False,"[[-8.585676, 41.148522], [-8.585712, 41.148639...",150
1,T2,B,NaN,57.0,20000108,1408038611,A,False,"[[-8.610876, 41.14557], [-8.610858, 41.145579]...",585
2,T3,B,NaN,15.0,20000370,1408038568,A,False,"[[-8.585739, 41.148558], [-8.58573, 41.148828]...",585
3,T4,B,NaN,53.0,20000492,1408039090,A,False,"[[-8.613963, 41.141169], [-8.614125, 41.141124...",105
4,T5,B,NaN,18.0,20000621,1408039177,A,False,"[[-8.619903, 41.148036], [-8.619894, 41.148036]]",15


In [34]:
start = []
destination = []
for i in range(320):
    start.append(test['POLYLINE'][i][0])
    destination.append(test['POLYLINE'][i][-1])
test['start'] = start    
test['destination'] = destination

In [35]:
test.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,duration,start,destination
0,T1,B,NaN,15.0,20000542,1408039037,A,False,"[[-8.585676, 41.148522], [-8.585712, 41.148639...",150,"[-8.585676, 41.148522]","[-8.584884, 41.146623]"
1,T2,B,NaN,57.0,20000108,1408038611,A,False,"[[-8.610876, 41.14557], [-8.610858, 41.145579]...",585,"[-8.610876, 41.14557]","[-8.601894, 41.163597]"
2,T3,B,NaN,15.0,20000370,1408038568,A,False,"[[-8.585739, 41.148558], [-8.58573, 41.148828]...",585,"[-8.585739, 41.148558]","[-8.574903, 41.167719]"
3,T4,B,NaN,53.0,20000492,1408039090,A,False,"[[-8.613963, 41.141169], [-8.614125, 41.141124...",105,"[-8.613963, 41.141169]","[-8.614638, 41.14098]"
4,T5,B,NaN,18.0,20000621,1408039177,A,False,"[[-8.619903, 41.148036], [-8.619894, 41.148036]]",15,"[-8.619903, 41.148036]","[-8.619894, 41.148036]"


In [29]:
type(test.POLYLINE[0])

list

In [8]:
op1.head()

,datapoints,timeinterval
0,11.0,165.0
1,40.0,600.0
2,40.0,600.0
3,8.0,120.0
4,2.0,30.0


# 8888

In [12]:
df = df.drop(['ORIGIN_CALL', 'ORIGIN_STAND'], axis = 1).head()

In [13]:
df.head()

,TRIP_ID,CALL_TYPE,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."


In [8]:
df.TAXI_ID.unique()

array([20000589, 20000596, 20000320, 20000520, 20000337, 20000231,
       20000456, 20000011, 20000403, 20000233, 20000571, 20000497,
       20000570, 20000005, 20000089, 20000423, 20000657, 20000309,
       20000161, 20000178, 20000235, 20000653, 20000009, 20000648,
       20000424, 20000010, 20000372, 20000686, 20000435, 20000154,
       20000060, 20000167, 20000503, 20000621, 20000463, 20000612,
       20000360, 20000574, 20000173, 20000560, 20000492, 20000112,
       20000305, 20000004, 20000620, 20000671, 20000341, 20000015,
       20000307, 20000171, 20000201, 20000007, 20000199, 20000611,
       20000398, 20000067, 20000569, 20000473, 20000367, 20000672,
       20000190, 20000600, 20000496, 20000013, 20000454, 20000534,
       20000195, 20000406, 20000325, 20000101, 20000632, 20000308,
       20000333, 20000481, 20000486, 20000450, 20000545, 20000665,
       20000012, 20000688, 20000540, 20000436, 20000517, 20000153,
       20000458, 20000577, 20000105, 20000128, 20000616, 20000

In [7]:
len(df.TAXI_ID.unique())

444

# 8888

In [10]:
missing_values_count = df.isnull().sum()
missing_values_count

TRIP_ID               0
CALL_TYPE             0
ORIGIN_CALL     1340415
ORIGIN_STAND     899986
TAXI_ID               0
TIMESTAMP             0
DAY_TYPE              0
MISSING_DATA          0
POLYLINE              0
dtype: int64